# CS579: Lecture 23  
*[Dr. Aron Culotta](http://cs.iit.edu/~culotta)*  
*[Illinois Institute of Technology](http://iit.edu)*

**Final Review**

- l10 -> end
- sentiment analysis
  - lexicons
  - wordnet
  - how to combine words with different sentiment?
- classification
  - workflow (collect, label, train, test, etc)
  - computing feature vectors
  - generalization error
  - variants of SimpleMachine
  - computing cross-validation accuracy (why?)
  - bias vs. variance
- demographics
  - pitfalls of using name lists
  - computing odds ratio
  - smoothing
  - ngrams
  - tokenization
  - stop words
  - regularization (why?)
- logistic regression, linear regression
  - no need to do calculus, but do you understand the formula?
  - apply classification function given data/parameters
  - what does the gradient represent?
- feature representation
  - tf-idf
  - csr_matrix: how does this data structure work? (data, column index, row pointer)
- recommendation systems
  - content-based
    - tf-idf
    - cosine similarity
  - collaborative filtering
    - user-user
    - item-item
    - measures: jaccard, cosine, pearson. Why choose one over another
  - How to compute the recommended score for a specific item?
- k-means
  - compute cluster assignment, means, and error function
  - what effect does k have?


### Note on a3

In [2]:
import pandas as pd
movies = pd.DataFrame([[123, ['horror', 'horror', 'romance', 'romance', 'romance']],
                       [456, ['romance']]],
                      columns=['movieId', 'tokens'])
movies

,movieId,tokens
0,123,"[horror, horror, romance, romance, romance]"
1,456,[romance]


In [3]:
# querying with pandas.
import numpy as np
# movies.sort_values('movieId', ascending=False)
movies[(movies['movieId'] == 123) | (movies['movieId'] < 500)]

,movieId,tokens
0,123,"[horror, horror, romance, romance, romance]"
1,456,[romance]


**tf-idf**  
traditional definition:

$$tfidf(i, d) = \frac{tf(i, d)}{max_k tf(k, d)} * log10(N/df(i))$$

- df(horror) = 1
- df(romance) = 2

- tf (horror, 123) = 2
- tfidf(horror, 123) = 2 / 3 * log10(2/1)

In [16]:
# dot products.
from scipy.sparse import csr_matrix
a = csr_matrix([1, 2, 3, 0, 0, 0, 5])
b = csr_matrix([4, 5, 6, 0, 0, 0, 0])
a.dot(b.T).sum()

32

In [5]:
(a + b).todense()

matrix([[5, 7, 9, 0, 0, 0, 5]], dtype=int64)

In [18]:
print(a.todense())
print(a.T.todense())
print(a.dot(b.T).sum())

[[1 2 3 0 0 0 5]]
[[1]
 [2]
 [3]
 [0]
 [0]
 [0]
 [5]]
32


In [7]:
# NOT this!
a.todense().dot(b.T.todense()).sum()

32

In [8]:
a.shape

(1, 7)

In [9]:
# And definitely not this:
total = 0
for i in range(a.shape[1]):
    total += a[0, i] * b[0, i]
total

32

** How to construct a csr_matrix? **

https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

We'll use:

```
csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]
```

In [22]:
# why modifying a csr_matrix is slow:
m = csr_matrix([[1,2,0,0,0,3], [4,5,6,0,0,0]])
print('indices:', m.indices)  # columns of each non-zero value
print('indptr:', m.indptr)   # index into m.indices/m.data for start of each row
print('data:', m.data)     # non-zero values
m[0,2] = 10
print('\nsetting (0,2) to 10\n')
print('indices:', m.indices)
print('indptr:', m.indptr)
print('data:', m.data)

indices: [0 1 5 0 1 2]
indptr: [0 3 6]
data: [1 2 3 4 5 6]

setting (0,2) to 10

indices: [0 1 2 5 0 1 2]
indptr: [0 4 7]
data: [ 1  2 10  3  4  5  6]


/usr/local/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [26]:
# what does a lil_matrix look like?
from scipy.sparse import lil_matrix
m = lil_matrix([[1,2,0,0,0,3], [4,5,6,0,0,0]])
print('data:', m.data)
print('rows:', m.rows)
print('\nsetting (0,2) to 10\n')
m[0,2] = 10
print('data:', m.data)
print('rows:', m.rows)
print(m.todense())

data: [list([1, 2, 3]) list([4, 5, 6])]
rows: [list([0, 1, 5]) list([0, 1, 2])]

setting (0,2) to 10

data: [list([1, 2, 10, 3]) list([4, 5, 6])]
rows: [list([0, 1, 2, 5]) list([0, 1, 2])]
[[ 1  2 10  0  0  3]
 [ 4  5  6  0  0  0]]


In [27]:
m = lil_matrix((2, 10))
# for loop; update m[i,j] = x
m[0,2] = 10
m.tocsr()

<2x10 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [28]:
m = csr_matrix((5, 8))
m[3, 5] = 10
m.indices
m.indptr
m.data

/usr/local/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


array([ 10.])

In [14]:
from collections import Counter
import math
def featurize(movies):
    dfs = Counter()
    data = []
    rows = []
    cols = []
    # set document frequencies
    for tokens in movies['tokens']:
        dfs.update(set(tokens))  # note set! 
                                 # count each term once per doc
        # dfs += set(tokens)  # will create a new copy...slow
    vocab = {v: i for i, v in enumerate(sorted(dfs))}
    N = len(movies)
    n_cols = len(vocab)
    vectors = []  # list of csr_matrices, one per movie
    for tokens in movies.tokens:
        tfs = Counter(tokens)
        maxtf = max(tfs.values())
        rows = [0] * len(tfs)  # row indices are all 0
        cols = [vocab[t] for t in tfs] # col indices are feautre indices
        # values are tf-idf: tf(i, d) / max_k tf(k, d) * log10(N/df(i))
        data = [v / maxtf * math.log10(N / dfs[t])
                for t, v in tfs.items()]
        print('tf:', [v / maxtf for t, v in tfs.items()])
        print('idf_weight:', [math.log10(N / dfs[t]) for t, v in tfs.items()])

        # construct the matrix for this movie
        vectors.append(csr_matrix((data, (rows, cols)),
                                   shape=(1, n_cols)))
        
    movies['features'] = vectors
    return movies, vocab

movies, vocab = featurize(movies)
movies

tf: [0.6666666666666666, 1.0]
idf_weight: [0.3010299956639812, 0.0]
tf: [1.0]
idf_weight: [0.0]


,movieId,tokens,features
0,123,"[horror, horror, romance, romance, romance]","(0, 0)\t0.200686663776\n (0, 1)\t0.0"
1,456,[romance],"(0, 1)\t0.0"


In [15]:
movies['features'][0].todense()

matrix([[ 0.20068666,  0.        ]])

In [31]:
movies['features'][0].dot(movies['features'][0].T).sum()

0.040275137017536239

<br><br><br><br><br><br><br>
## Privacy and Ethics in OSNA

<br><br><br><br><br><br><br>
<br><br><br><br><br><br><br>


![harvard](images/harvard.png)
[source](https://www.chronicle.com/article/Harvards-Privacy-Meltdown/128166)
- A study to measure homophily in Facebook
  - Do friends have similar tastes in books/music/etc.
- Questions about data collection process
- Easy to de-anonymize
> ... a privacy scholar at the University of Wisconsin at Milwaukee, Michael Zimmer, showed that the "anonymous" data of Mr. Kaufman and his colleagues could be cracked to identify the source as Harvard undergraduates.

<br><br><br><br><br><br>

![fb](images/fb.png)
[source](https://www.nytimes.com/2014/06/30/technology/facebook-tinkers-with-users-emotions-in-news-feed-experiment-stirring-outcry.html)
- A study by Facebook to measure how your feed affects your mood
  - Manipulated the feed of ~600k users by altering number of positive/negative posts the user sees
  - Measured the positive/negative posts the user then wrote
> Although academic protocols generally call for getting people’s consent before psychological research is conducted on them, Facebook didn’t ask for explicit permission from those it selected for the experiment. It argued that its 1.28 billion monthly users gave blanket consent to the company’s research as a condition of using the service.

<br><br><br><br><br><br>

![rec](images/rec.png)
[source](https://spectrum.ieee.org/tech-talk/telecom/internet/women-less-likely-to-be-shown-ads-for-highpaying-jobs)
- Fairness in recommendation systems
- Experiment: create fake Google profiles, then search for jobs
- Only difference was gender
> The male profiles were much more likely to be shown ads for a career coaching service for executive positions paying over $200,000. The Google ad network showed this ad to the male users more than 1800 times, but only about 300 times to women.


<br><br><br><br><br><br>


![pro](images/pro.png)
<img src="images/housing.png" width=400>
[source](https://www.propublica.org/article/facebook-advertising-discrimination-housing-race-sex-national-origin)
- Facebook let you advertise apartment rentals while excluding people based on ethnicity

> Last week, ProPublica bought dozens of rental housing ads on Facebook, but asked that they not be shown to certain categories of users, such as African Americans, mothers of high school kids, people interested in wheelchair ramps, Jews, expats from Argentina and Spanish speakers.  
All of these groups are protected under the federal Fair Housing Act, which makes it illegal to publish any advertisement “with respect to the sale or rental of a dwelling that indicates any preference, limitation, or discrimination based on race, color, religion, sex, handicap, familial status, or national origin.” Violators can face tens of thousands of dollars in fines.  
**Every single ad was approved within minutes.**


<br><br><br><br><br><br>

Many ethical issues in online social network analysis:

- **Privacy**: What data can be collected? shared? with whom?
- **Consent**: Scientists are bound by ethical guidelines when conducting research involving humans.
- **Fairness**: When algorithms are used to make decisions about people, they must not discriminate based on protected classes of people (e.g., by gender, race/ethnicity, sexual orientation, etc.).

**How can we decide if our behavior is ethical?**

<br><br><br>

**[Belmont Report](https://en.wikipedia.org/wiki/Belmont_Report)**
- Ethical guidelines published in 1979
- Response to notorious [Tuskegee syphilis experiments](https://en.wikipedia.org/wiki/Tuskegee_syphilis_experiment)
  - 40 year study by U.S. Public Health Service on population of impoverished African Americans in Alabama
  > Researchers knowingly failed to treat patients appropriately after the 1940s validation of penicillin was found as an effective cure for the disease they were studying.
  
**Three ethical principals outlines in Belmont Report**

1. **Respect for persons:** protecting the autonomy of all people and treating them with courtesy and respect and allowing for informed consent.
  - Researchers must be truthful and conduct no deception
  - Subjects enter into the research voluntarily and with adequate information  
<br><br>
2. **Beneficence:** The philosophy of "Do no harm" while maximizing benefits for the research project and minimizing risks to the research subjects
<br><br>
3. **Justice:** ensuring reasonable, non-exploitative, and well-considered procedures are administered fairly — the fair distribution of costs and benefits to potential research participants — and equally.

Applying the above principals leads to the following requirements:

1. **Informed Consent:**  Respect for persons requires that subjects, to the degree that they are capable, be given the opportunity to choose what shall or shall not happen to them. 
<br><br>
2. **Assessment of Risks and Benefits:** We must protect against risk of harm to subjects and also consider loss of the substantial benefits that might be gained from research.
  - Are the expected benefits of the research worth the cost?
<br><br>  
3. **Selection of Subjects:** Potentially beneficial research should not be restricted only to some subjects; nor should only "undesirable" persons be selected for risky research.


<br><br><br><br>
** Institutional Review Board:**
- Every university has a panel of faculty that assess a proposed research project by the above criteria
- Much of social media research can be deemed "non human subjects" research if passively observing public data:
  > Research involving collection or study of existing data, documents, records, pathological specimens, or diagnostic specimens: (i) if these sources are publicly available; or (ii) if the information is recorded by the investigator in such a manner that subjects cannot be identified, directly or through identifiers linked to the subjects

<br><br><br><br>
**Challenges in applying these principals to OSNA**

- Do users know which data is private?
![privacy](images/privacy.png)
[source](https://www.usnews.com/opinion/articles/2012/12/28/is-facebooks-privacy-policy-too-confusing)
> The family photo posted by Zuckerberg showed up on the newsfeed of Vox Media's Callie Schweitzer because they have a mutual friend, but Zuckerberg did not intend for the photo to become public. She said it was "way uncool" for Schweitzer to post the private photo to Twitter.

<br><br><br>

- How can we ensure anonymity?
![aol](images/aol.png)
[source](https://techcrunch.com/2006/08/06/aol-proudly-releases-massive-amounts-of-user-search-data/)
>  AOL has released very private data about its users without their permission. While the AOL username has been changed to a random ID number, the abilitiy to analyze all searches by a single user will often lead people to easily determine who the user is, and what they are up to. The data includes personal names, addresses, social security numbers and everything else someone might type into a search box.


<br><br><br>
- How can we ensure fairness?
![kleinberg](images/kleinberg.png)
> Recent discussion in the public sphere about algorithmic classification has involved tension between competing notions of what it means for a probabilistic classification to be fair to different groups. We formalize three fairness conditions that lie at the heart of these debates, and we prove that except in highly constrained special cases, there is no method that can satisfy these three conditions simultaneously. 


**Conclusion**
- Think carefully before conducting studies involving human data
- Would a reasonable person be upset about how the research was conducted?
- Have Institutional Review Board review research proposal
- Get feedback from others working in the area